In [1]:
import train_test_generation
from keras.layers import Dense,Conv2D,MaxPooling2D,UpSampling2D
from keras import Input, Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import regularizers

In [2]:
data = train_test_generation.generateDataset(10**5, 'raw')

In [3]:
data.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,...,cold_resistance,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots
0,13/01/2022,1,54,27,18,59,126.171041,191.197972,0,38.786593,...,70.0,True,True,False,False,False,True,False,False,False
1,12/01/2022,23,75,76,38,19,72.584686,146.704262,1,39.440707,...,90.0,False,False,False,False,False,False,True,True,False
2,11/01/2022,24,79,21,24,34,94.628970,171.240883,0,30.344912,...,100.0,False,False,False,False,False,False,False,False,False
3,10/01/2022,-4,35,1,13,56,100.132516,157.595977,1,55.860029,...,70.0,False,True,True,False,True,True,False,False,False
4,09/01/2022,-12,42,3,8,44,85.532023,168.084876,1,41.048939,...,80.0,False,False,True,True,True,True,False,False,False


In [4]:
user_params_df = data[['age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi']]

In [5]:
user_params_df.head()

,age,weight,height,sex,fatpercentage,bmi
0,59,126.171041,191.197972,0,38.786593,34.513827
1,19,72.584686,146.704262,1,39.440707,33.725589
2,34,94.628970,171.240883,0,30.344912,32.270760
3,56,100.132516,157.595977,1,55.860029,40.316691
4,44,85.532023,168.084876,1,41.048939,30.274116


In [6]:
#https://www.analyticsvidhya.com/blog/2021/06/complete-guide-on-how-to-use-autoencoders-in-python/

In [7]:
#num = user_params_df.values
#num = tf.convert_to_tensor(num)

In [8]:
from sklearn.preprocessing import MinMaxScaler 
X = MinMaxScaler().fit_transform(user_params_df)

In [9]:
#https://www.geeksforgeeks.org/ml-classifying-data-using-an-auto-encoder/

In [10]:
input_layer = Input(shape =(X.shape[1], ))

In [20]:
#building the autoencoder network
encoded = Dense(100, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(25, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(12, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(6, activation ='linear')(encoded)
  
# Building the Decoder network
decoded = Dense(12, activation ='linear')(encoded)
decoded = Dense(25, activation ='linear')(decoded)
decoded = Dense(50, activation ='linear')(decoded)
decoded = Dense(100, activation ='linear')(decoded)
  
# Building the Output Layer
output_layer = Dense(X.shape[1], activation ='relu')(decoded)
output_layer = Dense(X.shape[1], activation ='relu')(decoded)

In [21]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer ="adam", loss ="binary_crossentropy" , metrics=['accuracy'])

In [28]:
autoencoder.fit(X, X, 
                batch_size = 20, epochs = 25, 
                shuffle = True, validation_split = 0.20)

Epoch 1/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5243 - accuracy: 0.9603 - val_loss: 0.5239 - val_accuracy: 0.9474
Epoch 2/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5770 - accuracy: 0.9249 - val_loss: 0.5225 - val_accuracy: 0.9837
Epoch 3/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5238 - accuracy: 0.9664 - val_loss: 0.5223 - val_accuracy: 0.9822
Epoch 4/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5218 - accuracy: 0.9756 - val_loss: 0.5228 - val_accuracy: 0.9775
Epoch 5/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5220 - accuracy: 0.9748 - val_loss: 0.5217 - val_accuracy: 0.9843
Epoch 6/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5232 - accuracy: 0.9701 - val_loss: 0.5218 - val_accuracy: 0.9840
Epoch 7/25
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5229 - accuracy: 0.9697 - val_loss: 0.5224 - val_accuracy:

In [29]:
encoded

<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'dense_37')>